<a href="https://colab.research.google.com/github/UEPP40/PUM/blob/friska_test/2505moviesPUM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd


ratings = pd.read_csv('https://s3-us-west-2.amazonaws.com/recommender-tutorial/ratings.csv')
movies = pd.read_csv('https://s3-us-west-2.amazonaws.com/recommender-tutorial/movies.csv')


totalratings = ratings.shape[0]
totalmovies = movies.shape[0]

totalusers = ratings['userId'].nunique()

avgratings_per_user = totalratings / totalusers

avgratings_per_movie = totalratings / totalmovies


ratingsdistribution = ratings['rating'].value_counts().sort_index()


print("All ratings count:", totalratings)
print("All movies count:", totalmovies)
print("All users count", totalusers)
print("Average rating per user:", avgratings_per_user)
print("Average rating per movie:", avgratings_per_movie)
print("\nRatings distribution:")
print(ratingsdistribution)


All ratings count: 100836
All movies count: 9742
All users count 610
Average rating per user: 165.30491803278687
Average rating per movie: 10.350646684459043

Ratings distribution:
rating
0.5     1370
1.0     2811
1.5     1791
2.0     7551
2.5     5550
3.0    20047
3.5    13136
4.0    26818
4.5     8551
5.0    13211
Name: count, dtype: int64


In [3]:
movie_titles = dict(zip(movies['movieId'], movies['title']))
m = ratings.groupby('movieId')['rating'].agg(['count', 'mean'])
m = m['count'].mean()
C = ratings['rating'].mean()


def bayesian_avg(ratings):
    bayesian_avg = (C * m + ratings.sum()) / (C + ratings.count())
    return bayesian_avg

bayesian_avgratings = ratings.groupby('movieId')['rating'].agg(bayesian_avg).reset_index()
bayesian_avgratings.columns = ['movieId', 'bayesian_avg']

movie_stats = movies.merge(bayesian_avgratings, on='movieId')

print(movie_stats.head())

   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  bayesian_avg  
0  Adventure|Animation|Children|Comedy|Fantasy      4.024276  
1                   Adventure|Children|Fantasy      3.645857  
2                               Comedy|Romance      3.708193  
3                         Comedy|Drama|Romance      5.028823  
4                                       Comedy      3.558189  


In [4]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix

ratings = pd.read_csv('https://s3-us-west-2.amazonaws.com/recommender-tutorial/ratings.csv')
movies = pd.read_csv('https://s3-us-west-2.amazonaws.com/recommender-tutorial/movies.csv')
totalmovies = movies.shape[0]
totalusers = ratings['userId'].nunique()
totalratings = ratings.shape[0]

avgratings_per_user = totalratings / totalusers

avgratings_per_movie = totalratings / totalmovies


ratings_distribution = ratings['rating'].value_counts().sort_index()

print("All ratings count:", totalratings)
print("All movies count:", totalmovies)
print("All users count", totalusers)
print("Average rating per user:", avgratings_per_user)
print("Average rating per movie:", avgratings_per_movie)
print("\nRatings distribution:")
print(ratingsdistribution)
movie_titles = dict(zip(movies['movieId'], movies['title']))

C = ratings['rating'].mean()
m = ratings.groupby('movieId')['rating'].agg(['count', 'mean'])
m = m['count'].mean()


def bayesian_avg(ratings):
    bayesian_avg = (C * m + ratings.sum()) / (C + ratings.count())
    return bayesian_avg


bayesian_avg_ratings = ratings.groupby('movieId')['rating'].agg(bayesian_avg).reset_index()
bayesian_avg_ratings.columns = ['movieId', 'bayesian_avg']


movie_stats = movies.merge(bayesian_avg_ratings, on='movieId')


print(movie_stats.head())

def create_sparse(df):
    M = df['userId'].nunique()
    N = df['movieId'].nunique()
    user_mapper = dict(zip(np.unique(df["userId"]), list(range(M))))
    movie_mapper = dict(zip(np.unique(df["movieId"]), list(range(N))))
    user_inv_mapper = dict(zip(list(range(M)), np.unique(df["userId"])))
    movie_inv_mapper = dict(zip(list(range(N)), np.unique(df["movieId"])))
    user_index = [user_mapper[i] for i in df['userId']]
    item_index = [movie_mapper[i] for i in df['movieId']]
    X = csr_matrix((df["rating"], (user_index, item_index)), shape=(M,N))
    return X, user_mapper, movie_mapper, user_inv_mapper, movie_inv_mapper

X, user_mapper, movie_mapper, user_inv_mapper, movie_inv_mapper = create_sparse(ratings)

n_total = X.shape[0] * X.shape[1]
n_ratings = X.nnz
sparsity = n_ratings / n_total
sparsity_p = round(sparsity * 100, 2)

print(f"Matrix sparsity: {sparsity_p}%")


All ratings count: 100836
All movies count: 9742
All users count 610
Average rating per user: 165.30491803278687
Average rating per movie: 10.350646684459043

Ratings distribution:
rating
0.5     1370
1.0     2811
1.5     1791
2.0     7551
2.5     5550
3.0    20047
3.5    13136
4.0    26818
4.5     8551
5.0    13211
Name: count, dtype: int64
   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  bayesian_avg  
0  Adventure|Animation|Children|Comedy|Fantasy      4.024276  
1                   Adventure|Children|Fantasy      3.645857  
2                               Comedy|Romance      3.708193  
3                         Comedy|Drama|Romance      5.028823  
4                                       

In [5]:
from sklearn.neighbors import NearestNeighbors
import numpy as np

def find_similar_movies(movie_id, X, movie_mapper, movie_inv_mapper, k, metric='cosine'):
    X = X.T
    neighbour_ids = []
    movie_ind = movie_mapper[movie_id]
    movie_vec = X[movie_ind]
    if isinstance(movie_vec, (np.ndarray)):
        movie_vec = movie_vec.reshape(1, -1)
    kNN = NearestNeighbors(n_neighbors=k+1, algorithm="brute", metric=metric)
    kNN.fit(X)
    neighbour = kNN.kneighbors(movie_vec, return_distance=False)
    neighbour = neighbour[0][1:]
    for n in neighbour:
        neighbour_ids.append(movie_inv_mapper[n])
    return neighbour_ids

recommendation = find_similar_movies(1, X, movie_mapper, movie_inv_mapper, k=10)


for i, movie_id in enumerate(recommendation, 1):
    print(f"{i}. {movie_titles[movie_id]}")


1. Toy Story 2 (1999)
2. Jurassic Park (1993)
3. Independence Day (a.k.a. ID4) (1996)
4. Star Wars: Episode IV - A New Hope (1977)
5. Forrest Gump (1994)
6. Lion King, The (1994)
7. Star Wars: Episode VI - Return of the Jedi (1983)
8. Mission: Impossible (1996)
9. Groundhog Day (1993)
10. Back to the Future (1985)


In [6]:

!pip install surprise
from surprise.model_selection import cross_validate
from surprise import SVD
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import train_test_split
from surprise import accuracy

max_rating = ratings.rating.max()
min_rating = ratings.rating.min()

reader = Reader(rating_scale=(min_rating, max_rating))
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
algo = SVD()

#Cross validation olega na podziale danych na kilka części (zwanych „folds”) w celu sprawdzenia, jak dobrze model będzie generalizował na nowych, niewidzialnych danych
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 3.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357240 sha256=a0e829dd8ff7588ec8b7d875a3192a28630ae2daa6b292099e9625e446997932
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise
Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8707  0.8752  0.8659  0.8769  0.8742  0.8726  0.0039  
MAE (testset)     0.6684  0.6707  0.6663  0.6731  0.6724  0.6702  0.0025  
Fit time          1.49    1.55    1.62    2.38    1.49    1.70    0.34    
Test time         0.24    0.13    0.33    0.12    0.23    0.21    0.08    


{'test_rmse': array([0.87068952, 0.87524261, 0.86593923, 0.8768913 , 0.87419967]),
 'test_mae': array([0.66843934, 0.67072997, 0.66626442, 0.67305271, 0.67236401]),
 'fit_time': (1.489121913909912,
  1.5482096672058105,
  1.6154491901397705,
  2.375845193862915,
  1.489720106124878),
 'test_time': (0.2436845302581787,
  0.12940263748168945,
  0.33483052253723145,
  0.11871600151062012,
  0.2337024211883545)}

In [7]:

trainset, testset = train_test_split(data, test_size=0.25)

algorithm = SVD()

algorithm.fit(trainset)
predictions = algorithm.test(testset)

accuracy.rmse(predictions)

RMSE: 0.8695


0.8694534408795721

In [8]:
from surprise.prediction_algorithms import SVD, KNNBasic, KNNBaseline
from surprise import Dataset
from surprise.model_selection import cross_validate, GridSearchCV
from surprise import Reader
from surprise import accuracy, Dataset, SVD
from surprise.model_selection import train_test_split
algorithm = KNNBasic()
algorithm.fit(trainset)
predict = algorithm.test(testset)
accuracy.rmse(predict)


Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9459


0.9459237129234362

In [9]:
param_grid = {
    'k': [1, 2, 5, 10, 20, 30, 40, 50]
}

gs_knn = GridSearchCV(KNNBasic, param_grid, measures=['rmse'], cv=5, n_jobs=-1)
gs_knn.fit(data)

In [10]:
gs_knn.best_params

cross_validate(KNNBasic(), data, measures=['RMSE', 'MAE'], cv=5, verbose=True)


Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9448  0.9458  0.9468  0.9426  0.9554  0.9471  0.0044  
MAE (testset)     0.7248  0.7255  0.7251  0.7225  0.7306  0.7257  0.0027  
Fit time          0.11    0.13    0.13    0.13    0.13    0.13    0.01    
Test time         1.32    1.32    1.48    1.32    1.46    1.38    0.07    


{'test_rmse': array([0.94482931, 0.94583568, 0.94678919, 0.94255682, 0.95542385]),
 'test_mae': array([0.72482688, 0.72554744, 0.72506027, 0.72252496, 0.73062406]),
 'fit_time': (0.10578632354736328,
  0.13210535049438477,
  0.13216495513916016,
  0.12810754776000977,
  0.12741518020629883),
 'test_time': (1.319908857345581,
  1.3211071491241455,
  1.4794209003448486,
  1.3214361667633057,
  1.464327335357666)}

In [13]:
algorithm = KNNBaseline()
algorithm.fit(trainset)
predictions = algorithm.test(testset)
accuracy.rmse(predict)

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9459


0.9459237129234362